In [1]:
from experiment import *
from experiment import BasicCNN


# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = BasicCNN()
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
criterion = nn.BCELoss() #TODO - loss criteria are defined in the torch.nn package

#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters(), lr = 0.0001) #TODO - optimizers are defined in the torch.optim package

CUDA is supported
Model on CUDA? True


In [2]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []

# Begin training procedure
for epoch in range(num_epochs):

    N = 50
    N_minibatch_loss = 0.0    

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):

        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        labels1=torch.tensor(labels, dtype=torch.long, device=computing_device)

        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(images)
        v_o,i_o=torch.max(outputs,1)
        v_l, i_l=torch.max(labels1,1)
        loss = criterion(outputs,labels)
        
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
        
        #TODO: Implement cross-validation
        
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

    print("Finished", epoch + 1, "epochs of training")
print("Training complete after", epoch, "epochs")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 1, average minibatch 0 loss: 0.015
Epoch 1, average minibatch 50 loss: 0.729
Epoch 1, average minibatch 100 loss: 0.721
Epoch 1, average minibatch 150 loss: 0.715
Epoch 1, average minibatch 200 loss: 0.713
Epoch 1, average minibatch 250 loss: 0.712
Epoch 1, average minibatch 300 loss: 0.707
Epoch 1, average minibatch 350 loss: 0.705
Epoch 1, average minibatch 400 loss: 0.702
Epoch 1, average minibatch 450 loss: 0.699
Epoch 1, average minibatch 500 loss: 0.694
Epoch 1, average minibatch 550 loss: 0.693
Epoch 1, average minibatch 600 loss: 0.690
Epoch 1, average minibatch 650 loss: 0.691
Epoch 1, average minibatch 700 loss: 0.685
Epoch 1, average minibatch 750 loss: 0.684
Epoch 1, average minibatch 800 loss: 0.681
Epoch 1, average minibatch 850 loss: 0.678
Epoch 1, average minibatch 900 loss: 0.678
Epoch 1, average minibatch 950 loss: 0.673
Epoch 1, average minibatch 1000 loss: 0.669
Epoch 1, average minibatch 1050 loss: 0.670
Epoch 1, average minibatch 1100 loss: 0.666
Epoch 1, av

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [ ]:
def accuracy(outputs, labels):
    o=(outputs>0.5).float()
    return torch.sum(o==labels,0).float()/labels.shape[0]

def true_positive(outputs, labels):
    o=(outputs>0.5).float()
    return torch.sum(o+labels ==2.,0)

def false_positive(outputs, labels):
    o=(outputs>0.5).float()
    return torch.sum((labels-o)<0,0)

def false_negative(outputs, labels):
    o=(outputs>0.5).float()
    return torch.sum((o-labels)<0,0)

def precision(outputs, labels):
    return (true_positive(outputs, labels))/(true_positive(outputs, labels)+false_positive(outputs, labels))

def recall(outputs, labels):
    return (true_positive(outputs, labels))/(true_positive(outputs, labels)+false_negative(outputs, labels))

def BCR(outputs, labels):
    return (precision(outputs, labels)+recall(outputs, labels))/2